In [ ]:
from spacy.matcher.dependencymatcher import defaultdict
%matplotlib inline

### Parse and store SNOMED taxonomy

In [ ]:
from rdflib.extras.external_graph_libs import *
from rdflib import Graph
import networkx as nx
import pathlib

In [ ]:
def node_to_id(node):
    return node.split('/')[-1]

In [ ]:
path = pathlib.Path("./data/snomed_taxonomy.rdf")

# every node with 'http://snomed.info/id/SNOMED_ID' is child of 'http://www.w3.org/2002/07/owl#Class'
# 'http://snomed.info/id/138875005' is top node --> SNOMED CT Concept (SNOMED RT+CTV3)
# graph is directed in top direction (that is, a neighbor is/are only the direct parent(s))

graph = Graph()
graph.parse(path, format="application/rdf+xml")
nx_graph = rdflib_to_networkx_multidigraph(graph)

id_node_mapping = {}
for n in list(nx_graph.nodes()):
    _id = node_to_id(n)
    # Remove meta nodes (e.g. 'owl#Class') and top level node 'SNOMED CT Concept (SNOMED RT+CTV3)'
    if (not _id.isnumeric()) or (node_to_id(n) == "138875005"):
        nx_graph.remove_node(n)
        continue
    id_node_mapping[_id] = n

In [ ]:
def id_to_node(snomed_id: str, mapping_dict: dict = id_node_mapping):
    node = mapping_dict.get(snomed_id, None)
    return node

In [ ]:
def parents_of_id(snomed_id: str, mapping_dict: dict = id_node_mapping):
    for neighbor in nx_graph.neighbors(id_to_node(snomed_id, mapping_dict)):
        _id = node_to_id(neighbor)
        if not _id.isnumeric():
            continue
        yield _id

In [ ]:
def children_of_id(snomed_id: str, mapping_dict: dict = id_node_mapping):
    for predecessor in nx_graph.predecessors(id_to_node(snomed_id, mapping_dict)):
        _id = node_to_id(predecessor)
        if not _id.isnumeric():
            continue
        yield _id

---
### Read SNOMED Interface Terminology

In [ ]:
import collections

In [ ]:
interface_terminology_path = pathlib.Path("./data/SCT-GIT_de_drugs_nlp.dat")
interface_terminology = collections.defaultdict(list)

for line in interface_terminology_path.open('r', encoding='utf-8').readlines():
    # first name in the resulting list for each entry is the preferred name
    snomed_id, internal_id, concept_name, interface_name = line.strip().split("\t")
    interface_terminology[snomed_id].append(interface_name)
first_level_concepts = [node_to_id(n) for n in nx_graph.nodes() if
                        nx_graph.out_degree(n) == 0 and node_to_id(n) in interface_terminology]

In [ ]:
len(interface_terminology)

In [ ]:
len(id_node_mapping)

In [ ]:
len(first_level_concepts)

---
### Create/upload concepts to TOP FW

In [ ]:
import requests
import uuid
import pickle
import itertools
from typing import Union
from tqdm.autonotebook import tqdm

In [ ]:
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(itertools.islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [58]:
dummy_parent_ids = {}
def create_dummy_parent(stratum: int, dummy_ids: dict):
    # We need categories for each stratum that can contain concepts which have a parent that isn't represented in the interface terminology
    if stratum not in dummy_ids:
        _id = str(uuid.uuid4())
        dummy_ids[stratum] = _id
    else:
        _id = dummy_ids[stratum]
        
    return_dict = {
        "entityType": "single_concept",
        "id": dummy_ids[stratum],
        "titles": [
            {
                "lang": "de",
                "text": f"Dummy Category for Stratum {stratum}"
            }
        ],
    }
    if stratum > 0:
        return_dict["superConcepts"] = [
            {
                "entityType": "single_concept",
                "id": dummy_ids.get(stratum - 1)
            }
        ]
    return return_dict

def create_top_fw_concept(
        concept_id: str,
        concept_id_store: dict,
        top_id_store: dict,
        dummy_ids: dict,
        stratum: int,
        parent_id: Union[str, list] = None
):
    if concept_id not in top_id_store:
        _id = str(uuid.uuid4())
        top_id_store[concept_id] = _id
    else:
        _id = top_id_store[concept_id]
        
    _data = {
        "entityType": "single_concept",
        "id": _id,
        "titles": [
            {
                "lang": "de",
                "text": concept_id_store[concept_id][0]
            }
        ],
        "codes": [
            {
                "uri": f"http://snomed.info/id/{concept_id}",
                "codeSystem": {
                    "uri": "http://snomed.info/id",
                    "shortName": "SNOMED CT"
                },
                "code": concept_id,
            }
        ] 
    }
    
    if len(concept_id_store[concept_id]) > 1:
        _data["synonyms"] = [
            {
                "lang": "de",
                "text": syn
            } for syn in concept_id_store[concept_id][1:]
        ]
        
    if parent_id is not None:
        parent_id = [parent_id] if isinstance(parent_id, str) else parent_id
        add_list = []
        for pid in parent_id:
            if not (pid in concept_id_store and pid in top_id_store):
                continue
            add_list.append(
                {
                    "entityType": "single_concept",
                    "id": top_id_store.get(pid)
                }
            )
        if len(add_list) > 0:
            _data["superConcepts"] = add_list
        else:
            _data["superConcepts"] = [
                {
                    "entityType": "single_concept",
                    "id": dummy_ids.get(stratum - 1)
                }
            ]
    
    return _data

In [ ]:
def post_to_top_fw(post_data: Union[list, dict], post_api_token: str, organisation: str = "imise", repository: str = "snomed_interface_terminology"):
    _endpoint_suffix = "" if isinstance(post_data, dict) else "/bulk"
    request = requests.post(
        url=f"https://top.imise.uni-leipzig.de/api/{organisation}/{repository}/entity{_endpoint_suffix}",
        headers={"Authorization": "Bearer " + post_api_token},
        json=post_data
    )
    return request

In [ ]:
username = input("Please give your TOP FW username: ")
password = input("Please give your TOP FW password: ")
api_token = requests.post(
    url = "https://top.imise.uni-leipzig.de/auth/realms/top-realm/protocol/openid-connect/token",
    data = {
        "client_id": "top-frontend",
        "username": username,
        "password": password,
        "grant_type": "password"
    }
)

In [59]:
pickle_path = "./data/snomed_concept_to_top_id.pickle"
snomed_concept_to_top_id = pickle.load(pathlib.Path(pickle_path).open('rb')) if pathlib.Path(pickle_path).exists() else {}

In [60]:
post_array = defaultdict(list)
with tqdm(total=len(nx_graph)) as pbar:
    for i, stratum in enumerate(nx.topological_generations(nx_graph.reverse(copy=True))):
        pbar.set_description_str(f"Strata {str(i).zfill(2)}: ")
        post_array[i].append(create_dummy_parent(i, dummy_parent_ids))
        for node in stratum:
            pbar.update(1)
            _node_concept_id = node_to_id(node)
            if _node_concept_id not in interface_terminology:
                continue
            _parents = None
            if i > 0:
                _parents = [node_to_id(x) for x in nx_graph[node]]
            post_array[i].append(
                create_top_fw_concept(_node_concept_id, interface_terminology, snomed_concept_to_top_id, dummy_parent_ids, i, _parents)
            )
#pickle.dump(snomed_concept_to_top_id, pathlib.Path(pickle_path).open('wb'))

  0%|          | 0/366789 [00:00<?, ?it/s]

In [61]:
[x for x in post_array[1] if "Dummy" in x["titles"][0]["text"]]

[{'entityType': 'single_concept',
  'id': '58d3c952-f8bb-415f-9441-5abacda26b82',
  'titles': [{'lang': 'de', 'text': 'Dummy Category for Stratum 1'}],
  'superConcepts': [{'entityType': 'single_concept',
    'id': 'f333aa6f-8de3-4a95-942f-f4e1a885307f'}]}]

In [62]:
[x for x in post_array[0] if "f333aa6f-8de3-4a95-942f-f4e1a885307f" in x["id"]]

[{'entityType': 'single_concept',
  'id': 'f333aa6f-8de3-4a95-942f-f4e1a885307f',
  'titles': [{'lang': 'de', 'text': 'Dummy Category for Stratum 0'}]}]

In [ ]:
stratum = 1
batch_size = 100
with tqdm(total=len(post_array[stratum])) as pbar:
    for batch in batched(post_array[stratum], batch_size):
        post_to_top_fw(list(batch), api_token.json()['access_token'])
        pbar.update(batch_size)

In [ ]:
for stratum_n in range(5, len(post_array)):
    print(f"-- Stratum {stratum_n} --")
    for entity in tqdm(post_array[stratum_n]):
        post_to_top_fw(entity, api_token.json()['access_token'])

-- Stratum 5 --


  0%|          | 0/29739 [00:00<?, ?it/s]